In [ ]:
# huggingface-cli download meta-llama/Meta-Llama-3-8B --include "original/*" --local-dir Meta-Llama-3-8B
# 
import transformers
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

class LlamaTextGenerator:
    def __init__(self, model_path):
        self.model_path = model_path
        self.tokenizer = None
        self.model = None
        self.pipeline = None

    def load_model(self):
        # トークナイザーの読み込み
        self.tokenizer = LlamaTokenizer.from_pretrained(self.model_path)

        # モデルの読み込み
        # model = AutoModelForCausalLM.from_pretrained("path", torch_dtype = "auto"
        self.model = LlamaForCausalLM.from_pretrained(
            self.model_path,
            torch_dtype=torch.bfloat16,
            low_cpu_mem_usage=True,
            device_map="auto"
        )

        # パイプラインの作成
        self.pipeline = transformers.pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

    def generate_text(self, prompt, max_length=50, num_return_sequences=1):
        if self.pipeline is None:
            raise ValueError("Model is not loaded. Please call load_model() first.")
        
        result = self.pipeline(prompt, max_length=max_length, num_return_sequences=num_return_sequences)
        return result[0]['generated_text']

def main():
    # モデルのパスを設定
    model_path = "./Meta-Llama-3-8B/original"

    # LlamaTextGeneratorのインスタンスを作成
    text_generator = LlamaTextGenerator(model_path)

    # モデルを読み込む
    text_generator.load_model()

    # テキスト生成の実行
    prompt = "Hey how are you doing today?"
    generated_text = text_generator.generate_text(prompt)

    print(generated_text)

if __name__ == "__main__":
    main()

In [7]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.float16}, device_map="auto")
pipeline("Hey how are you doing today?")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


NotImplementedError: The operator 'aten::isin.Tensor_Tensor_out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.